<img style="float: right;" src="./img/es2_logo_tiny.jpg"></img>


<h1 align="center">Point Extraction application to calculate OC4 Chlrophyll algorithm</h1>
<h2 align="center">Application Generating OC4 Chlorophyll values from reflectance data</h2>
<hr>


### Background

<p>The OC4 chlorophyll algorith takes the ratio of either ['412','490','443', '510'] divided by 555 nm relectance data. The band used is based on the biggest value created, so the ability to quickly test each band is a huge benefit</p>

### Basic imports and setup the query templates

In [1]:
import requests
import numpy as np


query = '''for a in (CCI_V2_monthly_Rrs_555), b in (CCI_V2_monthly_Rrs_{rrs_band})
return
encode (
log(b[Lat({lat}), Long({lon}), ansi("2003-06-30T23:59:00Z")] / a[Lat({lat}), Long({lon}), ansi("2003-06-30T23:59:00Z")])
, "csv" )'''

chl_query = '''for a in (CCI_V2_monthly_chlor_a)
return
encode (
a[Lat({lat}), Long({lon}), ansi("2003-06-30T23:59:00Z")]
, "csv" )'''

### Create a helper function to generate R value
<p>We also return from this function the OCX chlorophyll value as this is the algorithm currently used by OC-CCI. We use this value to analyses the difference between the two algortihms</p>

In [8]:
def run_both(lat,lon, r_band):
   data = requests.post("http://earthserver.pml.ac.uk/rasdaman/ows/wcps",data={"query":query.format(lat=lat, lon=lon, rrs_band=r_band)} )

   chl_data = requests.post("http://earthserver.pml.ac.uk/rasdaman/ows/wcps",data={"query":chl_query.format(lat=lat, lon=lon)} ).text[1:-1]

   R = data.text[1:-1]
   R = float(R)

   oc4 = pow(10, 0.366 - 3.067*R + 1.930*pow(R,2) + 0.649*pow(R,3) - 1.532*pow(R,4))
   ocx = float(chl_data)
   return (oc4,ocx,(oc4-ocx),R)


### The actual science
<p>This is the actual running, we test which of the selection of bands produces the biggest R value, we then use them each in a OC4 run and print out teh difference between OC4 and OCX</p>

In [9]:
collector = []
bands = ['412','490','443', '510']


for i, j in zip(np.linspace(30, 40, 10).tolist(), np.linspace(-10, -20, 10).tolist()):
   print ("-"*20)
   for band in bands:
      print ("testing band "+band)
      lat = round(i,2)
      lon = round(j,2)
      t_vals = run_both(lat,lon, band)
      print ("R value for {band} @ Lat:{lat} Long:{long} = {R}".format(band=band,lat=lat,long=lon,R=t_vals[3]))
      collector.append(t_vals)
      print ("OC4 Value {}".format(t_vals[0]))
      print ("the difference between OC4 and OCx (current CCI algorithm)")
      print (sum(abs(x[2]) for x in collector if x[2] > -20) / len(collector))



--------------------
testing band 412
R value for 412 @ Lat:30.0 Long:-10.0 = 0.043416209317233
OC4 Value 1.723963800636994
the difference between OC4 and OCx (current CCI algorithm)
1.5373338006369939
testing band 490
R value for 490 @ Lat:30.0 Long:-10.0 = 0.114403305419354
OC4 Value 1.0992449018592643
the difference between OC4 and OCx (current CCI algorithm)
1.224974351248129
testing band 443
R value for 443 @ Lat:30.0 Long:-10.0 = 0.075051908432192
OC4 Value 1.4025321894670755
the difference between OC4 and OCx (current CCI algorithm)
1.2219502973211112
testing band 510
R value for 510 @ Lat:30.0 Long:-10.0 = 0.0827853840082016
OC4 Value 1.3354283416750394
the difference between OC4 and OCx (current CCI algorithm)
1.2036623084095932
--------------------
testing band 412
R value for 412 @ Lat:31.11 Long:-11.11 = 0.269227384351834
OC4 Value 0.4839367216411611
the difference between OC4 and OCx (current CCI algorithm)
0.9759381910559067
testing band 490
R value for 490 @ Lat:31.11 Lo

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>